# Keypoints clean-up after PoseAnnotator - For Velocity extraction

Azilis EVEN

In [11]:
import os
import json
import shutil
import zipfile

In [12]:
# For a folder

parent = "C:/Users/Azilis/Documents/master-thesis-ipynb/season18-19/velocity-estimation/2018-12-21_orebro-skelleftea2"
dir_str = os.path.join(parent, "keypoints-str")
if not os.path.exists(dir_str):
    os.mkdir(dir_str)
dir_float = os.path.join(parent, "keypoints")
if not os.path.exists(dir_float):
    os.mkdir(dir_float)

## CleanUp from pose annotator zip
for filename in os.listdir(parent):
    file = os.path.join(parent, filename)
    # checking if it is a zip file
    if filename.endswith(".zip"):
        number = filename.split(".")[0]
        newfile = "image"+number+"_keypoints.json"
        #print(newfile)
        with zipfile.ZipFile(file) as z:
            with z.open(newfile) as zf, open(os.path.join(dir_str, newfile), 'wb') as f:
                shutil.copyfileobj(zf, f)
    
for filename in os.listdir(dir_str):
    file = os.path.join(dir_str, filename)
    # checking if it is a file
    if os.path.isfile(file):
        print(filename)
        with open(file) as f:
            keypoints = json.load(f)
        #print(keypoints)
        nbPeople=0
        zeroIndexes = []
        for person in keypoints['people']:
            nbPeople+=1
            allZero = True
            for element in person:
                # Check for str after pose-annotator
                for i in range(len(person[element])):
                    if float(person[element][i]) != 0.0 and i%3 !=2 :
                        allZero = False
                    if isinstance(person[element][i], str):
                        person[element][i] = float(person[element][i])
                # Check for confidence values at 0 after pose-annotator
                for i in range(0, len(person[element])-1, 3):
                    if person[element][i]!=0 and person[element][i+1]!=0 and person[element][i+2]==0:
                        person[element][i+2] = 0.9
            if allZero:
                zeroIndexes.append(nbPeople-1)
                nbPeople-=1
        print(nbPeople)
        # Warning Ugly removal, only works since reading and deleting in order
        for index in zeroIndexes:
            keypoints['people'].pop(index)
            for i in range(index+1, len(zeroIndexes)):
                zeroIndexes[i]-=1
            print("Removed empty person")
        if len(person["face_keypoints_2d"]) == 0:
            person["face_keypoints_2d"] = [0]*210
        if len(person["hand_left_keypoints_2d"]) == 0:
            person["hand_left_keypoints_2d"] = [0]*63
        if len(person["hand_right_keypoints_2d"]) == 0:
            person["hand_right_keypoints_2d"] = [0]*63
        #print(keypoints)
        save = dir_float + "/" + filename
        with open(save, "w") as f:
            json.dump(keypoints, f, indent=2)

image00_keypoints.json
2
image01_keypoints.json
2
image02_keypoints.json
2
image03_keypoints.json
2
image04_keypoints.json
2
image05_keypoints.json
2
image06_keypoints.json
2
